In [ ]:
# %pip install redis

In [ ]:
import orjson
import pandas as pd
import redis

dev = True
if dev:
    h = "prefect-reseng-dev-redis-svc.prefect-dev"
    p = 6379
else:
    h = "localhost"
    p = 6579

r = redis.Redis(host=h, port=p, decode_responses=True)
print(r.info())

data = r.xrevrange(name="events", count=1000)
data.reverse()

ee = []
for d in data:
    dd = orjson.loads(d[1]["data"])
    if dd["event"].startswith("prefect.flow-run"):
        ee.append(dd)
print(len(ee))

xx = {}
start = pd.Timestamp("2025-11-11T12:09:02.052803Z")
for e in ee:
    if pd.Timestamp(e["occurred"]) < start:
        continue
    st = e["resource"]["prefect.state-type"]
    na = e["resource"]["prefect.resource.name"]
    if na not in xx:
        xx[na] = []
    xx[na].append(st)
print(len(xx))

for k, v in xx.items():
    print("%20s %s" % (k, v))